# Introduction

This notebook contains a data analysis project related to traffic count on the German freeway system. This is a purely recreational project that I created with the intent of learning data analysis methodologies with a practical example.

The data used in this notebook was obtained from the "Bundesanstalt für Straßenwesen" webiste, www.bast.de. The website provides hourly data on all automated traffic counting stations on the German freeway system ("Autobahn"). This is used as the raw data for this project. The raw data file is farily large at around 1.6GB, which is an additional challenge in this analysis.

The data used is from 2018, because at this moment the 2019 data was not available yet. However, this notebook can easily be changed to work with more recent data later.

The focus of this workbook is data exploration and visualization. Since the data is provided, the sourcing data part of this project is very simple and easy. Also, I will not be using the data to make any kinds of prediction (at this point in time at least)

So this project focused on the these main activities:

    - Analysis of the structural elements of the data provided
    - Cleaning of the raw data, modifying the data to meet the needs of the analysis
    - Analysing and visualizing interesting aspects of the data

The following cell contains code that we need to rerun when we come back to this notebook. The rest of the cell only needs to be run, if the content shall be executed again.

In [1]:
import pandas as pd
import os

year = "2018"

Note: In all code cells that have code that takes long to execute or performs a task that should not be repeated, there is a variable called breaker that is set to True as the default value. If such a cell is to be executed, then breaker needs to be set to False manually. This is done to not have cells rerun that take a long time to execute.

## Section 1: Structural analysis of the data

### Step 1.1: Size and shape of the dataset

The data is provided as a .txt file, so first of all I am checking the format of the data in the file, to see if the data is in a usable CSV-type fromat, that can easily be read in.

In [2]:
filename = year+"_A_S.txt"
file = open(filename, 'r')
number_of_lines = 5
line_number = 0

for line in file:
    print(line)
    line_number += 1
    if line_number >= number_of_lines: break;
        
file.close()

TKNR;Zst;Land;Strklas;Strnum;Datum;Wotag;Fahrtzw;Stunde;KFZ_R1;K_KFZ_R1;KFZ_R2;K_KFZ_R2;Lkw_R1;K_Lkw_R1;Lkw_R2;K_Lkw_R2;PLZ_R1;K_PLZ_R1;Pkw_R1;K_Pkw_R1;Lfw_R1;K_Lfw_R1;Mot_R1;K_Mot_R1;PmA_R1;K_PmA_R1;Bus_R1;K_Bus_R1;LoA_R1;K_LoA_R1;Lzg_R1;K_Lzg_R1;Sat_R1;K_Sat_R1;Son_R1;K_Son_R1;PLZ_R2;K_PLZ_R2;Pkw_R2;K_Pkw_R2;Lfw_R2;K_Lfw_R2;Mot_R2;K_Mot_R2;PmA_R2;K_PmA_R2;Bus_R2;K_Bus_R2;LoA_R2;K_LoA_R2;Lzg_R2;K_Lzg_R2;Sat_R2;K_Sat_R2;Son_R2;K_Son_R2

5329;4382;16;A;  73;180101; 1;s;01;   11;-;    7;-;    1;-;    1;-;    9;-;    9;-;    0;-;    0;-;    0;-;    0;-;    1;-;    0;-;    0;-;    1;-;    6;-;    5;-;    1;-;    0;-;    0;-;    0;-;    1;-;    0;-;    0;-;    0;-

5329;4382;16;A;  73;180101; 1;s;02;   27;-;   33;-;    1;-;    1;-;   25;-;   23;-;    2;-;    0;-;    0;-;    0;-;    1;-;    0;-;    0;-;    1;-;   31;-;   29;-;    2;-;    0;-;    0;-;    0;-;    0;-;    1;-;    1;-;    1;-

5329;4382;16;A;  73;180101; 1;s;03;   18;-;   27;-;    1;-;    1;-;   17;-;   17;-;    0;-;    0;-;    

By looking at the first 5 lines of the file I can see that the data is provided in a CSV-line format, using ";" as the separator. Therefore I should be able to import the data directly into da dataframe. 

In [8]:
#This variable is used to block the code execution if "execute all" should be selected. To run this code, breaker needs to be set to false manually
breaker = True

if breaker == False:
    file = open(filename, 'r')
    number_of_lines = sum(1 for line in file)
    print("The file contains " + str(number_of_lines) +" lines!")
    file.close()
else:
    print("Code not executed, because breaker is set!")

Code not executed, because breaker is set!


I just counted the lines in the source file. It is pretty much, therefore I decided to split the file into monthly data before starting to work with the data.

### Step 1.2: Importing the data into a dataframe and reviewing the dataframe using the head method

Because the raw data is so huge, in the following I will split the data into monthly packages that can be analysed separately. Anything that I do later then needs to be done in a loop over 12 monthly files, however this should be much easier on memory usage and works much faster during the development process.

If needed, I will create yearly dataframes with only the data I need for yearly anaylses later, that cannot be composed of the monthly data.

In [8]:
#obtain the headline, that will be needed in all the subfiles
file = open(filename, 'r')
headline = file.readline()

#from checking the first few lines earlier, it can easily be seen that the date is the 6th entry in each line
#this element needs to be split to obtain the month information, it has the format YYMMDD

#create monthly files if they don't exist

for i in range(1,13):
    if os.path.isfile(year+"_"+str(i)+".txt"):
        print("File for month "+str(i)+" already exists!")
        continue
    file = open(year+"_"+str(i)+".txt", "w+")
    file.write(headline)
    file.close()
    
print("Files created!")

File for month 1 already exists!
File for month 2 already exists!
File for month 3 already exists!
File for month 4 already exists!
File for month 5 already exists!
File for month 6 already exists!
File for month 7 already exists!
File for month 8 already exists!
File for month 9 already exists!
File for month 10 already exists!
File for month 11 already exists!
File for month 12 already exists!
Files created!


In [5]:
#i start with month january (01).
#i print a progress statement every 100000 line to see that the script is running (it may take a while)

current_month = 0
line_count = 0
file = open(filename, 'r')
headline = file.readline()
subfile = open(year+"_1.txt")
#This variable is used to block the code execution if "execute all" should be selected. To run this code, breaker needs to be set to false manually
breaker = True

if breaker == False:
    for line in file:
        splitted_line = line.split(";")
        month = int(splitted_line[5][2:4])
        #if the month is different than the month before, the file is closed and the appropriate file is opened
        if not int(month) == current_month:
            subfile.close()
            subfile = open(year+"_"+str(month)+".txt", "a")
            current_month = int(month)
        subfile.write(line)
        line_count += 1
        if line_count % 100000 == 0: print(str(line_count) + " lines processed!             \r")

    subfile.close()
    print("")
    print("File split completed!")
    
else:
    print("Code not executed, because breaker is set!")

Code not executed, because breaker is set!


Now I will take a look at the january file manually to get a better understanding of the data I am dealing with. In order to do that, I am importing the data into a dataframe.

In [6]:
df = pd.read_csv(year+"_1.txt", sep=";")
df.head()

,TKNR,Zst,Land,Strklas,Strnum,Datum,Wotag,Fahrtzw,Stunde,KFZ_R1,...,Bus_R2,K_Bus_R2,LoA_R2,K_LoA_R2,Lzg_R2,K_Lzg_R2,Sat_R2,K_Sat_R2,Son_R2,K_Son_R2
0,5329,4382,16,A,73,180101,1,s,1,11,...,0,-,1,-,0,-,0,-,0,-
1,5329,4382,16,A,73,180101,1,s,2,27,...,0,-,0,-,1,-,1,-,1,-
2,5329,4382,16,A,73,180101,1,s,3,18,...,1,-,0,-,0,-,0,-,1,-
3,5329,4382,16,A,73,180101,1,s,4,22,...,0,-,2,-,0,-,0,-,0,-
4,5329,4382,16,A,73,180101,1,s,5,16,...,0,-,1,-,0,-,0,-,1,-


In [7]:
print("The dataframe has "+str(len(df.index))+" rows!")
del df

The dataframe has 629424 rows!


The monthly file is still quite big, but we should be able to work with it quite well. There are 57 columns in the dataframe, of which we probably don't need all. We will take a closer look at the colums we need later.

### Step 1.3: Check for non-exisiting values (nan)

As a first step, I will check if there are any missing values in the data, which we need to fix first.

In [3]:
#This variable is used to block the code execution if "execute all" should be selected. To run this code, breaker needs to be set to false manually
breaker = True

if breaker == False:
    for month in range (1,13):
        df = pd.read_csv(""+year+"_"+str(month)+".txt", sep=";")
        has_nan = df.isnull().values.any()
        if has_nan:
            print("Data for month " + str(month) + " has nan values!")
        else:
            print("Data for month " + str(month) + " has NO nan values!")
        del df
    
    print("nan check complete!")
else:
    print("Code not executed, because breaker is set!")

Data for month 1 has NO nan values!
Data for month 2 has NO nan values!
Data for month 3 has NO nan values!
Data for month 4 has NO nan values!
Data for month 5 has NO nan values!
Data for month 6 has NO nan values!
Data for month 7 has NO nan values!
Data for month 8 has NO nan values!
Data for month 9 has NO nan values!
Data for month 10 has NO nan values!
Data for month 11 has NO nan values!
Data for month 12 has NO nan values!
nan check complete!


OK good, no nan values that I need to deal with for the moment. That was sort of expected, as the source of the data states, that missing data has already been estimated before publishing, however it is always good to make sure and check.

### Step 1.4: Available data

Finally, I want to find out how many sample point (traffic counting points) are reflected in the data and how many motoways have any sample points at all.

In [9]:
dataframe_result = pd.DataFrame(columns=["sample_points","motorways"])
    
print(dataframe_result)
print("Grouping complete!")
del dataframe_result

Empty DataFrame
Columns: [sample_points, motorways]
Index: []
Grouping complete!
